In [1]:
import pickle as pkl
from transformer_raw.modules import encoder, decoder
def load(path : str) -> None:
	pickle_encoder = open(f'{path}/encoder.pkl', 'rb')
	pickle_decoder = open(f'{path}/decoder.pkl', 'rb')
	# print(type(self.encoder.dropout.scale))

	encoder = pkl.load(pickle_encoder)
	decoder = pkl.load(pickle_decoder)
	pickle_encoder.close()
	pickle_decoder.close()
	return encoder, decoder


In [2]:
path = "./saved_models/seq2seq_model/10"
encoder, decoder = load(path)


In [3]:
encoder.layers[1].self_attention.K_linear.w

array([[ 0.05060973, -0.04148768,  0.01022771, ..., -0.04246638,
         0.03684648,  0.03228686],
       [ 0.02292799,  0.00066971,  0.04401567, ...,  0.04953277,
         0.00582354, -0.01646453],
       [-0.05573905, -0.00559234,  0.0030537 , ...,  0.02988779,
        -0.06115827, -0.04564564],
       ...,
       [ 0.02449939, -0.03783581,  0.04394244, ..., -0.00181951,
        -0.01485202, -0.02642046],
       [-0.00151512, -0.09452108,  0.07570018, ...,  0.04546585,
        -0.07477983, -0.02964973],
       [ 0.0266874 ,  0.03324678, -0.0059617 , ..., -0.04180157,
        -0.00888584, -0.03500656]], dtype=float32)

In [ ]:
import numpy as np
w = np.random.rand(10, 20)
a = np.random.rand(10, 20)


In [44]:
from client_side.utils import Utils
import tenseal as ts
utils = Utils("./keys")


In [45]:
a = np.random.rand(11, 256)
b = np.random.rand(256, 256)

In [47]:
a_ = utils.encrypt_matrix(a)

In [ ]:
b_ = utils.encrypt_matrix(b)

In [ ]:

x = a.dot(b)
x_ = a_.dot(b_)

True